# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f76a4c22af0>
├── 'a' --> tensor([[-0.4956, -0.7713,  1.2544],
│                   [ 0.0810, -0.6108,  0.5154]])
└── 'x' --> <FastTreeValue 0x7f76a5c87df0>
    └── 'c' --> tensor([[ 0.7582, -0.6045, -0.5322,  1.0866],
                        [ 0.6540,  0.6687, -1.3448, -1.5811],
                        [ 0.0756, -0.0610,  0.0466,  0.0403]])

In [4]:
t.a

tensor([[-0.4956, -0.7713,  1.2544],
        [ 0.0810, -0.6108,  0.5154]])

In [5]:
%timeit t.a

90.4 ns ± 0.0607 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f76a4c22af0>
├── 'a' --> tensor([[ 0.3970,  0.4543, -0.5377],
│                   [-0.2230,  1.1254,  2.5987]])
└── 'x' --> <FastTreeValue 0x7f76a5c87df0>
    └── 'c' --> tensor([[ 0.7582, -0.6045, -0.5322,  1.0866],
                        [ 0.6540,  0.6687, -1.3448, -1.5811],
                        [ 0.0756, -0.0610,  0.0466,  0.0403]])

In [7]:
%timeit t.a = new_value

101 ns ± 1.96 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4956, -0.7713,  1.2544],
               [ 0.0810, -0.6108,  0.5154]]),
    x: Batch(
           c: tensor([[ 0.7582, -0.6045, -0.5322,  1.0866],
                      [ 0.6540,  0.6687, -1.3448, -1.5811],
                      [ 0.0756, -0.0610,  0.0466,  0.0403]]),
       ),
)

In [10]:
b.a

tensor([[-0.4956, -0.7713,  1.2544],
        [ 0.0810, -0.6108,  0.5154]])

In [11]:
%timeit b.a

79.6 ns ± 1.23 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4826, -0.5862,  1.4000],
               [ 0.4979, -0.8709, -0.6002]]),
    x: Batch(
           c: tensor([[ 0.7582, -0.6045, -0.5322,  1.0866],
                      [ 0.6540,  0.6687, -1.3448, -1.5811],
                      [ 0.0756, -0.0610,  0.0466,  0.0403]]),
       ),
)

In [13]:
%timeit b.a = new_value

658 ns ± 1.45 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.06 µs ± 19.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.5 µs ± 15.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

195 µs ± 19.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

211 µs ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f75bb2caa90>
├── 'a' --> tensor([[[-0.4956, -0.7713,  1.2544],
│                    [ 0.0810, -0.6108,  0.5154]],
│           
│                   [[-0.4956, -0.7713,  1.2544],
│                    [ 0.0810, -0.6108,  0.5154]],
│           
│                   [[-0.4956, -0.7713,  1.2544],
│                    [ 0.0810, -0.6108,  0.5154]],
│           
│                   [[-0.4956, -0.7713,  1.2544],
│                    [ 0.0810, -0.6108,  0.5154]],
│           
│                   [[-0.4956, -0.7713,  1.2544],
│                    [ 0.0810, -0.6108,  0.5154]],
│           
│                   [[-0.4956, -0.7713,  1.2544],
│                    [ 0.0810, -0.6108,  0.5154]],
│           
│                   [[-0.4956, -0.7713,  1.2544],
│                    [ 0.0810, -0.6108,  0.5154]],
│           
│                   [[-0.4956, -0.7713,  1.2544],
│                    [ 0.0810, -0.6108,  0.5154]]])
└── 'x' --> <FastTreeValue 0x7f75bb2ca8b0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

39.3 µs ± 159 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f75bb2caaf0>
├── 'a' --> tensor([[-0.4956, -0.7713,  1.2544],
│                   [ 0.0810, -0.6108,  0.5154],
│                   [-0.4956, -0.7713,  1.2544],
│                   [ 0.0810, -0.6108,  0.5154],
│                   [-0.4956, -0.7713,  1.2544],
│                   [ 0.0810, -0.6108,  0.5154],
│                   [-0.4956, -0.7713,  1.2544],
│                   [ 0.0810, -0.6108,  0.5154],
│                   [-0.4956, -0.7713,  1.2544],
│                   [ 0.0810, -0.6108,  0.5154],
│                   [-0.4956, -0.7713,  1.2544],
│                   [ 0.0810, -0.6108,  0.5154],
│                   [-0.4956, -0.7713,  1.2544],
│                   [ 0.0810, -0.6108,  0.5154],
│                   [-0.4956, -0.7713,  1.2544],
│                   [ 0.0810, -0.6108,  0.5154]])
└── 'x' --> <FastTreeValue 0x7f75bb2cac10>
    └── 'c' --> tensor([[ 0.7582, -0.6045, -0.5322,  1.0866],
                        [ 0.6540,  0.6687, -1.3448, -1.5811],
                 

In [23]:
%timeit t_cat(trees)

37.9 µs ± 1.98 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

87.2 µs ± 4.8 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.7582, -0.6045, -0.5322,  1.0866],
                       [ 0.6540,  0.6687, -1.3448, -1.5811],
                       [ 0.0756, -0.0610,  0.0466,  0.0403]],
              
                      [[ 0.7582, -0.6045, -0.5322,  1.0866],
                       [ 0.6540,  0.6687, -1.3448, -1.5811],
                       [ 0.0756, -0.0610,  0.0466,  0.0403]],
              
                      [[ 0.7582, -0.6045, -0.5322,  1.0866],
                       [ 0.6540,  0.6687, -1.3448, -1.5811],
                       [ 0.0756, -0.0610,  0.0466,  0.0403]],
              
                      [[ 0.7582, -0.6045, -0.5322,  1.0866],
                       [ 0.6540,  0.6687, -1.3448, -1.5811],
                       [ 0.0756, -0.0610,  0.0466,  0.0403]],
              
                      [[ 0.7582, -0.6045, -0.5322,  1.0866],
                       [ 0.6540,  0.6687, -1.3448, -1.5811],
                       [ 0.0756, -0.0610,  0.0466,  0.0403]],

In [26]:
%timeit Batch.stack(batches)

98.9 µs ± 240 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.7582, -0.6045, -0.5322,  1.0866],
                      [ 0.6540,  0.6687, -1.3448, -1.5811],
                      [ 0.0756, -0.0610,  0.0466,  0.0403],
                      [ 0.7582, -0.6045, -0.5322,  1.0866],
                      [ 0.6540,  0.6687, -1.3448, -1.5811],
                      [ 0.0756, -0.0610,  0.0466,  0.0403],
                      [ 0.7582, -0.6045, -0.5322,  1.0866],
                      [ 0.6540,  0.6687, -1.3448, -1.5811],
                      [ 0.0756, -0.0610,  0.0466,  0.0403],
                      [ 0.7582, -0.6045, -0.5322,  1.0866],
                      [ 0.6540,  0.6687, -1.3448, -1.5811],
                      [ 0.0756, -0.0610,  0.0466,  0.0403],
                      [ 0.7582, -0.6045, -0.5322,  1.0866],
                      [ 0.6540,  0.6687, -1.3448, -1.5811],
                      [ 0.0756, -0.0610,  0.0466,  0.0403],
                      [ 0.7582, -0.6045, -0.5322,  1.0866],
                   

In [28]:
%timeit Batch.cat(batches)

181 µs ± 182 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

475 µs ± 17.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
